In [1]:
from __future__ import division
import glob
import re
import os,sys
from astropy.io import fits
from astropy.io.fits import getheader, getdata
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
from scipy.stats import chisquare as cs
import sep
from matplotlib.patches import Ellipse

### First the object list will be defined, organized, and seperated into categories

In [2]:
# Create a list of all of the fits file directories
obj_list = glob.glob('/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data/*.fits')

In [3]:
obj_list

['/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d1.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d10.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d100.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d101.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d102.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d103.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d104.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d105.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d106.fits',
 '/Users/hitom/Downloads/Academ

In [4]:
# This will organize the obj_list by the index of the fits file before separation
Organized_obj_list = ['' for x in range(len(obj_list))]
for j in range(len(Organized_obj_list)):
    try:
        n = int(obj_list[j][94:97])
    except:
        try:
            n = int(obj_list[j][94:96])
        except:
            n = int(obj_list[j][94:95])
    Organized_obj_list[n-1] = obj_list[j]
Organized_obj_list

['/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d1.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d2.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d3.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d4.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d5.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d6.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d7.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d8.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classes/Spring 2019/ASTR136/Star_Clusters_Project_data\\d9.fits',
 '/Users/hitom/Downloads/Academics/UCSC Classe

In [5]:
# Seperate the fits files by categories
Biases = {}
Blue_Domes = {}
Violet_Domes = {}
Darks = {}
IC1B = {}
IC1V = {}
IC2B = {}
IC2V = {}
IC3B = {}
IC3V = {}
IC4B = {}
IC4V = {}
IC5B = {}
IC7B = {}
IC7V = {}
IC8B = {}
IC8V = {}
IC9B = {}
IC9V = {}
for i in range(len(Organized_obj_list)):
    if(i<10):
        Biases[i] = Organized_obj_list[i]
    elif(10<=i<25):
        Blue_Domes[i-10] = Organized_obj_list[i]
    elif(25<=i<41):
        Violet_Domes[i-25] = Organized_obj_list[i]
    elif(41<=i<50):
        Darks[i-41] = Organized_obj_list[i]
    elif(50<=i<168):
        continue
    elif(168<=i<173):
        IC1B[i-168] = Organized_obj_list[i]
    elif(173<=i<178):
        IC1V[i-173] = Organized_obj_list[i]
    elif(178<=i<183):
        IC2B[i-178] = Organized_obj_list[i]
    elif(183<=i<188):
        IC2V[i-183] = Organized_obj_list[i]
    elif(188<=i<193):
        IC3B[i-188] = Organized_obj_list[i]
    elif(193<=i<198):
        IC3V[i-193] = Organized_obj_list[i]
    elif(198<=i<203):
        IC4B[i-198] = Organized_obj_list[i]
    elif(203<=i<208):
        IC4V[i-203] = Organized_obj_list[i]
    elif(208<=i<213):
        IC5B[i-208] = Organized_obj_list[i]
    elif(213<=i<219):
        continue
    elif(219<=i<224):
        IC7B[i-219] = Organized_obj_list[i]
    elif(224<=i<229):
        IC7V[i-224] = Organized_obj_list[i]
    elif(229<=i<234):
        IC8B[i-229] = Organized_obj_list[i]
    elif(234<=i<239):
        IC8V[i-234] = Organized_obj_list[i]
    elif(239<=i<244):
        IC9B[i-239] = Organized_obj_list[i]
    elif(244<=i<249):
        IC9V[i-244] = Organized_obj_list[i]
    elif(249<=i<251):
        continue
    elif(i==251):
        SDSSB = Organized_obj_list[i]
    elif(i==252):
        SDSSV = Organized_obj_list[i]

### Define a few functions to make handling the arrays easier

In [6]:
# This returns the header of the fits file
def header(array,index):
    header = fits.open(array[index])[0].header
    return header

In [7]:
# This returns the array of the fits file
def data(array,*args):
    index = 0
    for num in args:
        index = num
    try:
        data = fits.open(array[index])[0].data
    except:
        data = fits.open(array)[0].data
    return data

### Now the bias frames will be averaged to get the bias of the telescope

In [8]:
def Bias(array,filename):
    # First add all of the bias frames in a temporary array
    Temp = np.zeros(data(array,0).shape)
    for i in range(len(array)):
        Temp = data(array,i) + Temp
    # Now the temporary array will be averaged for the bias of the telescope
    Average_Bias = Temp[0:1024,0:1024]/len(array)
    # Now the bias frame will be saved as fits files according to the 'filename' input
    # Note: try and except is used so it still runs when a bias still exists
    try:
        hdu = fits.PrimaryHDU(Average_Bias)
        hdul = fits.HDUList([hdu])
        hdul.writeto(filename)
    except:
        pass
    return Average_Bias

In [9]:
Master_Bias = Bias(Biases,'UCO_Lick_Nickel_Telescope_Master_Bias.fits')
Master_Bias

array([[1085.1, 1083. , 1084.1, ..., 1083.5, 1104.8, 1102.5],
       [1084.2, 1085.1, 1080.8, ..., 1079. , 1103.9, 1105.4],
       [1084.1, 1080.9, 1087.9, ..., 1078.8, 1107.7, 1103.7],
       ...,
       [1086.6, 1083.3, 1082.6, ..., 1085.2, 1106.3, 1103.7],
       [1085.8, 1082.1, 1082.7, ..., 1084.2, 1102.6, 1106.4],
       [1087.3, 1080.2, 1076. , ..., 1082.2, 1105.1, 1100.2]])

### Next the dark frames will be reduced and combined.

In [10]:
def Reduce_Darks(array,filename):
    # First the darks will be reduced to per second images, given that they all have an exposure time of 30 seconds.
    # They will simultaneously be added into an array and combined.
    Temp = {}
    Reduced_Darks = np.zeros(Master_Bias.shape)
    ExpTime = 30 #seconds
    for i in range(len(array)):
        Temp[i] = data(array,i)[0:1024,0:1024] - Master_Bias
        Reduced_Darks = Temp[i]/ExpTime + Reduced_Darks
    # Now they will all be averaged into a single dark frame.
    Reduced_Dark = Reduced_Darks/len(array)
    # Now the reduced dark will be saved as fits files according to the 'filename' input
    try:
        hdu = fits.PrimaryHDU(Reduced_Dark)
        hdul = fits.HDUList([hdu])
        hdul.writeto('{0}.fits'.format(filename))
    except:
        pass
    return Reduced_Dark

In [11]:
Master_Dark = Reduce_Darks(Darks,'UCO_Lick_Nickel_Telescope_Master_Dark')
Master_Dark

array([[ 0.41148148,  0.44814815,  0.45592593, ...,  0.41666667,
         0.37703704,  0.39444444],
       [ 0.61185185,  0.52259259,  0.57333333, ...,  0.65925926,
         0.39962963,  0.33481481],
       [ 0.59666667,  0.60333333,  0.40333333, ...,  0.6437037 ,
         0.19518519,  0.64333333],
       ...,
       [ 0.3762963 ,  0.2937037 ,  0.55037037, ...,  0.34888889,
         0.09      ,  0.19148148],
       [ 0.22518519,  0.38555556,  0.27296296, ...,  0.0637037 ,
         0.16148148, -0.03555556],
       [ 0.00851852,  0.40444444,  0.35185185, ...,  0.23037037,
        -0.05148148, -0.04      ]])

### Now the dome flats will be reduced and combined.

In [12]:
def Reduce_Domes(array,filename,string):
    # First the domes will be reduced to per second images, given that they all have an exposure time of 30 seconds.
    # They will simultaneously be added into an array and combined.
    Temp = {}
    Reduced_Dome = np.zeros(Master_Bias.shape)
    if string=='Blue':
        ExpTime = 30 #seconds
    elif string=='Violet':
        ExpTime = 3 #seconds
    for i in range(len(array)):
        Temp[i] = data(array,i)[0:1024,0:1024] - Master_Bias
        Reduced_Dome = Temp[i] + Reduced_Dome - ExpTime*Master_Dark
    # Now the reduced dome will be normalized and saved.
    Master_Dome = Reduced_Dome[0:1024,0:1024]/len(array)
    try:
        hdu = fits.PrimaryHDU(Master_Dome)
        hdul = fits.HDUList([hdu])
        hdul.writeto('{0}.fits'.format(filename))
    except:
        pass
    return Master_Dome

In [13]:
Master_Blue_Dome = Reduce_Domes(Blue_Domes,'UCO_Lick_Nickel_Telescope_Master_Dome_B','Blue')
Master_Blue_Dome

array([[  457.75555556,   324.02222222,   406.02222222, ...,
        10086.46666667, 10405.28888889, 17754.26666667],
       [ 1243.71111111,   681.28888889,   717.73333333, ...,
        22929.08888889, 24032.77777778, 47424.95555556],
       [ 1427.93333333,   729.93333333,   752.46666667, ...,
        23181.28888889, 24471.51111111, 52997.86666667],
       ...,
       [  689.37777778,   309.35555556,   301.28888889, ...,
        13522.53333333, 12991.86666667, 24348.75555556],
       [  646.77777778,   309.46666667,   306.37777778, ...,
        12456.88888889, 11709.62222222, 20411.46666667],
       [  709.97777778,   355.4       ,   327.31111111, ...,
        12944.95555556, 11780.44444444, 18098.46666667]])

In [14]:
Master_Violet_Dome = Reduce_Domes(Violet_Domes,'UCO_Lick_Nickel_Telescope_Master_Dome_V','Violet')
Master_Violet_Dome

array([[  491.72805556,   363.53055556,   438.71972222, ...,
         8686.4375    ,  8958.19388889, 15283.87916667],
       [ 1316.71444444,   736.76972222,   744.355     , ...,
        19758.02222222, 20643.58861111, 40651.15805556],
       [ 1524.4225    ,   791.415     ,   786.64      , ...,
        20017.51888889, 21029.83944444, 45554.87      ],
       ...,
       [  805.45861111,   372.44388889,   377.81138889, ...,
        11209.56583333, 10797.9925    , 20295.66305556],
       [  754.39944444,   369.61833333,   360.41861111, ...,
        10316.04638889,  9709.47805556, 17001.83166667],
       [  806.42444444,   438.02416667,   403.00694444, ...,
        10417.04638889,  9467.30444444, 14684.1075    ]])

### Now the clusters will be reduced individually so they can be aligned in AstroImagej and then recombined.

In [15]:
def Reduce_Blue_Cluster(array,filename):
    # First the clusters will be reduced to per second images, given that they all have an exposure time of 30 seconds.
    Temp = {}
    Reduced_Clusters = {}
    Master_Clusters = {}
    ExpTime = 30 #seconds
    for i in range(len(array)):
        Temp[i] = data(array,i)[0:1024,0:1024] - Master_Bias
        Reduced_Clusters[i] = (Temp[i] - ExpTime*Master_Dark)/(Master_Blue_Dome/np.max(Master_Blue_Dome))
        Master_Clusters[i] = Reduced_Clusters[i]
    # Now the reduced clusters will be saved.
    try:
        for i in range(len(array)):
            hdu = fits.PrimaryHDU(Master_Clusters[i])
            hdul = fits.HDUList([hdu])
            hdul.writeto('{0}_{1}.fits'.format(filename,i))
    except:
        pass
    return Master_Clusters

In [16]:
def Reduce_Violet_Cluster(array,filename):
    # First the clusters will be reduced to per second images, given that they all have an exposure time of 30 seconds.
    Temp = {}
    Reduced_Clusters = {}
    Master_Clusters = {}
    ExpTime = 30 #seconds
    for i in range(len(array)):
        Temp[i] = data(array,i)[0:1024,0:1024] - Master_Bias
        Reduced_Clusters[i] = (Temp[i] - ExpTime*Master_Dark)/(Master_Violet_Dome/np.max(Master_Violet_Dome))
        Master_Clusters[i] = Reduced_Clusters[i]
    # Now the reduced clusters will be saved.
    try:
        for i in range(len(array)):
            hdu = fits.PrimaryHDU(Master_Clusters[i])
            hdul = fits.HDUList([hdu])
            hdul.writeto('{0}_{1}.fits'.format(filename,i))
    except:
        pass
    return Master_Clusters

In [17]:
#Reduce_Blue_Cluster(IC1B,'UCO_Lick_Nickel_Telescope_IC1B')
#Reduce_Blue_Cluster(IC2B,'UCO_Lick_Nickel_Telescope_IC2B')
#Reduce_Blue_Cluster(IC3B,'UCO_Lick_Nickel_Telescope_IC3B')
#Reduce_Blue_Cluster(IC4B,'UCO_Lick_Nickel_Telescope_IC4B')
#Reduce_Blue_Cluster(IC5B,'UCO_Lick_Nickel_Telescope_IC5B')
#Reduce_Blue_Cluster(IC7B,'UCO_Lick_Nickel_Telescope_IC7B')
#Reduce_Blue_Cluster(IC8B,'UCO_Lick_Nickel_Telescope_IC8B')
#Reduce_Blue_Cluster(IC9B,'UCO_Lick_Nickel_Telescope_IC9B')

In [18]:
#Reduce_Violet_Cluster(IC1V,'UCO_Lick_Nickel_Telescope_IC1V')
#Reduce_Violet_Cluster(IC2V,'UCO_Lick_Nickel_Telescope_IC2V')
#Reduce_Violet_Cluster(IC3V,'UCO_Lick_Nickel_Telescope_IC3V')
#Reduce_Violet_Cluster(IC4V,'UCO_Lick_Nickel_Telescope_IC4V')
#Reduce_Violet_Cluster(IC7V,'UCO_Lick_Nickel_Telescope_IC7V')
#Reduce_Violet_Cluster(IC8V,'UCO_Lick_Nickel_Telescope_IC8V')
#Reduce_Violet_Cluster(IC9V,'UCO_Lick_Nickel_Telescope_IC9V')

In [19]:
# After aligning the reduced images in AstroImagej
Aligned_Blues = glob.glob('/Users/hitom/ASTR136_Code/Aligned_Blues/*.fits')
Aligned_Violets = glob.glob('/Users/hitom/ASTR136_Code/Aligned_Violets/*.fits')

In [20]:
# This will combine the blue and violet images
def Combine_Images(array,filename,letter):
    Combined_Arrays = {}
    for i in range(9):
        Temp = {}
        if(letter=='V' and i==4):
            continue
        elif(i==5):
            continue
        elif(i>5):
            if(letter=='V'):
                for j in range(5):
                    Temp[j] = data(array[5*i+j-10])
                Combined_Arrays[i] = Temp[0] + Temp[1] + Temp[2] + Temp[3] + Temp[4]
                try:
                    hdu = fits.PrimaryHDU(Combined_Arrays[i])
                    hdul = fits.HDUList([hdu])
                    hdul.writeto('{0}_IC{1}{2}_Master.fits'.format(filename,i+1,letter))
                except:
                    pass
            else:
                for j in range(5):
                    Temp[j] = data(array[5*i+j-5])
                Combined_Arrays[i] = Temp[0] + Temp[1] + Temp[2] + Temp[3] + Temp[4]
                try:
                    hdu = fits.PrimaryHDU(Combined_Arrays[i])
                    hdul = fits.HDUList([hdu])
                    hdul.writeto('{0}_IC{1}{2}_Master.fits'.format(filename,i+1,letter))
                except:
                    pass
        else:
            for j in range(5):
                Temp[j] = data(array[5*i+j])
            Combined_Arrays[i] = Temp[0] + Temp[1] + Temp[2] + Temp[3] + Temp[4]
            try:
                hdu = fits.PrimaryHDU(Combined_Arrays[i])
                hdul = fits.HDUList([hdu])
                hdul.writeto('{0}_IC{1}{2}_Master.fits'.format(filename,i+1,letter))
            except:
                pass
    return Combined_Arrays

In [21]:
Combined_Blues = Combine_Images(Aligned_Blues,'UCO_Lick_Nickel_Telescope','B')
Combined_Violets = Combine_Images(Aligned_Violets,'UCO_Lick_Nickel_Telescope','V')

In [22]:
Combined_Blues

{0: array([[-2.32183477e+04, -2.08357109e+04, -2.32282227e+04, ...,
         -2.84580719e+02, -9.28971100e+01, -2.24253693e+01],
        [-2.01426992e+04, -1.78105938e+04, -1.76253027e+04, ...,
          1.15008669e+01,  3.56685944e+01,  3.25501366e+01],
        [-1.22179375e+04, -1.39521367e+04, -1.39338848e+04, ...,
          2.55960503e+01,  1.32070169e+01,  3.55854836e+01],
        ...,
        [-3.38268398e+04, -3.91834180e+04, -4.01943750e+04, ...,
         -3.24282593e+02, -1.67941406e+02, -5.79484863e+01],
        [-3.07411484e+04, -3.53903594e+04, -3.90842305e+04, ...,
         -2.73589508e+02, -1.14035225e+02, -4.41451492e+01],
        [-2.94251719e+04, -3.64353398e+04, -3.23035195e+04, ...,
         -3.61535095e+02, -2.34641953e+02, -4.51555939e+01]], dtype=float32),
 1: array([[-9.32341602e+03, -1.68577422e+04, -1.34015527e+04, ...,
         -2.18141846e+02, -3.03425293e+02, -1.15667694e+02],
        [-4.09987695e+03, -7.34089844e+03, -7.79537305e+03, ...,
         -1.05708

### Now the standard star images will be reduced

In [34]:
def Reduce_Standard_Star(array,filename,letter):
    # First the standard stars will be reduced
    ExpTime = 25 #seconds
    Temp = data(array)[0:1024,0:1024] - Master_Bias
    Reduced_Star = (Temp - Master_Dark*ExpTime)
    if letter=='B':
        Master_Star = Reduced_Star/(Master_Blue_Dome/np.max(Master_Blue_Dome))
    elif letter=='V':
        Master_Star = Reduced_Star/(Master_Violet_Dome/np.max(Master_Violet_Dome))
    # Now the reduced star will be saved.
    try:
        hdu = fits.PrimaryHDU(Reduced_Star)
        hdul = fits.HDUList([hdu])
        hdul.writeto('{0}.fits'.format(filename))
    except:
        pass
    return Reduced_Star

In [35]:
Master_SDSSB = Reduce_Standard_Star(SDSSB,'UCO_Lick_Nickel_Telescope_SDSSB','B')
Master_SDSSV = Reduce_Standard_Star(SDSSV,'UCO_Lick_Nickel_Telescope_SDSSV','V')

In [36]:
np.median(Master_SDSSB)

-10.966666666666663

In [37]:
np.median(Master_SDSSV)

21.892592592592585

In [38]:
def Correct_Blues(array1,array2,filename):
    Median_Correction = np.median(array2) - np.median(array1)
    full = np.full(array1.shape,Median_Correction)
    array_new = array1 + full
    try:
        hdu = fits.PrimaryHDU(array_new)
        hdul = fits.HDUList([hdu])
        hdul.writeto('{0}.fits'.format(filename))
    except:
        pass

In [39]:
Correct_Blues(Master_SDSSB,Master_SDSSV,'Corrected_UCO_Lick_Nickel_Telescope_SDSSB')
Correct_Blues(Combined_Blues[0],Combined_Violets[0],'Corrected_UCO_Lick_Nickel_Telescope_IC1B')
Correct_Blues(Combined_Blues[1],Combined_Violets[1],'Corrected_UCO_Lick_Nickel_Telescope_IC2B')
Correct_Blues(Combined_Blues[2],Combined_Violets[2],'Corrected_UCO_Lick_Nickel_Telescope_IC3B')
Correct_Blues(Combined_Blues[3],Combined_Violets[3],'Corrected_UCO_Lick_Nickel_Telescope_IC4B')
Correct_Blues(Combined_Blues[4],Combined_Violets[3],'Corrected_UCO_Lick_Nickel_Telescope_IC5B')
Correct_Blues(Combined_Blues[6],Combined_Violets[6],'Corrected_UCO_Lick_Nickel_Telescope_IC7B')
Correct_Blues(Combined_Blues[7],Combined_Violets[7],'Corrected_UCO_Lick_Nickel_Telescope_IC8B')
Correct_Blues(Combined_Blues[8],Combined_Violets[8],'Corrected_UCO_Lick_Nickel_Telescope_IC9B')

### Now the fits files will be shown and run object detection on. The standard star will be used to determine magnitude and color by using the standard star as a value such that counts/counts_SS = ratio for the magnitude. The color will be created by subtracting the magnitude from the standard star magnitude of 1.

### After the object detection, aperature photometry is run to find the fluxes of the objects that will be used to find the magnitudes of the objects.

In [ ]:
# This masks the region between 1025 and 1056 since this only contains bias
def masker(array):
    masker = np.ones(array.shape)
    array_2 = np.zeros((1024,1024))
    masker[0:1024,0:1024] = array_2
    masked_array = ma.array(array,mask=masker)
    return masked_array

In [ ]:
# Create a function to show the image
def imshow(array):
    # First the image will be masked
    fits_file = masker(array)
    # Now the images will be shown
    m, s = np.mean(fits_file), np.std(fits_file)
    plt.figure(figsize=(7,7))
    plt.imshow(fits_file, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
    plt.xlim(0,1024)
    plt.colorbar();

In [ ]:
imshow(Combined_Blues[0])

In [ ]:
def object_detector(array,detection_threshold):
    # This is the masked array
    Image = masker(array)
    
    # This will detect the objects in the image using a detection threshold of 1.5 sigma with sigma being the RMS
    objects = sep.extract(array,detection_threshold,err=np.std(Image))
    
    # Now the background subtracted image will be plotted
    fig, ax = plt.subplots(figsize=(7,7))
    m, s = np.mean(Image), np.std(Image)
    im = ax.imshow(Image, interpolation='nearest', cmap='gray',vmin=m-s, vmax=m+s, origin='lower')
    plt.xlim(0,1024)

    # plot an ellipse for each object
    for i in range(len(objects)):
        e = Ellipse(xy=(objects['x'][i], objects['y'][i]),width=6*objects['a'][i],\
                    height=6*objects['b'][i],angle=objects['theta'][i] * 180. / np.pi)
        e.set_facecolor('none')
        e.set_edgecolor('red')
        ax.add_artist(e)
        
    flux, fluxerr, flag = sep.sum_circle(array, objects['x'], objects['y'],
                                     3.0, err=np.std(Image), gain=1.0)
    # show the objects results:
    for i in range(len(objects)):
        print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

### Object detection on all of the Master images